In [ ]:
import pandas as pd
import glob
import intervaltree

In [ ]:
df_human = pd.read_csv('data/Dec20_2015_storm_human_selecteted.csv', parse_dates=['start_time', 'end_time'])
print(f'{len(df_human.index)} human selections')
df_human.head()

In [ ]:
tmp = []
for file in glob.glob('data/Dec20_2015_storm_F*.csv'):
    df = pd.read_csv(file, parse_dates=['start_time', 'end_time'])
    sats = []
    for _, row in df.iterrows():
        for i in range(11, 30):
            if f'F{i}' in row.file:
                sats.append(i)
                break
    df['sat'] = sats
    tmp.append(df)
    
df_algorithm = pd.concat(tmp).reset_index()
print(f'{len(df_algorithm.index)} algorithm selections')
df_algorithm.head()

In [ ]:
def get_trees(df):
    trees = {}
    for sat in set(df.sat.values):
        trees[sat] = intervaltree.IntervalTree()
        for _, row in df.iterrows():
            if row.sat == sat:
                trees[sat][row.start_time:row.end_time] = True
    return trees

In [ ]:
trees_human = get_trees(df_human)
trees_human[16]

In [ ]:
trees_algorithm = get_trees(df_algorithm)
trees_algorithm

In [ ]:
trees_algorithm[16]

In [ ]:
both = 0
alg_not_human = 0
human_not_alg = 0
for sat in trees_algorithm:
    for interval in trees_algorithm[sat]:
        res = trees_human[sat][interval.begin:interval.end]
        if res:
            both += 1
        else:
            alg_not_human += 1


tmp = []
for sat in trees_human:
    for interval in trees_human[sat]:
        res = trees_algorithm[sat][interval.begin:interval.end]
        if not res:
            human_not_alg += 1
            tmp.append((sat, interval.begin.isoformat()))

print('Human Total: %d' % sum(len(t) for t in trees_human.values())) 
print('Algorithm Total: %d' % sum(len(t) for t in trees_algorithm.values())) 
print()
print('In Both Human and Algorithm: %d' %both)
print('In Algorithm but not Human: %d' % alg_not_human)
print('In Human but not Algorithm: %d' % human_not_alg)

In [ ]:
for x in sorted(tmp):print(x)